In [1]:
import graphlab as gl

# 1 取数据

In [2]:
normal = gl.SFrame.read_csv('data/normal_examples.csv')
evil = gl.SFrame.read_csv('data/xssed.csv')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1500988895.log


This non-commercial license of GraphLab Create for academic use is assigned to xyntax@163.com and will expire on July 23, 2018.


Finished parsing file /Users/xy/workspace/ML/DL_for_xss/data/normal_examples.csv

Parsing completed. Parsed 100 lines in 0.49853 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


...

Finished parsing file /Users/xy/workspace/ML/DL_for_xss/data/xssed.csv

Parsing completed. Parsed 100 lines in 0.074102 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/xy/workspace/ML/DL_for_xss/data/xssed.csv

Parsing completed. Parsed 40637 lines in 0.090048 secs.

In [3]:
normal = normal.random_split(0.1,seed=0)
evil = evil.random_split(0.1,seed=0)

In [7]:
normal = gl.SFrame(normal)
evil = gl.SFrame(evil)

In [9]:
normal

X1
[{'param': 'cn_600022%2Ccn_600516%2Ccn_000002% ...
"[{'param':'_%3D1498591621808'}, ..."


# 2 画数据

In [4]:
normal.show()

AttributeError: 'tuple' object has no attribute 'show'

In [3]:
normal.head()

param
_%3D1498591621808
code%3Dzs_000001%2Czs_399001%2Czs_399006%26cb% ...
_%3D1498591951848%26list%3Dml_sh600030 ...
6053%26ri%3Dzb6-00f%7E-04gUry-01h- ...
b1498592370545%3D1
v%3D13111002
COLLCC%3D3442798258%26
t%3Dcheck%26rec%3Dstratus%26etyp%3Dconnect%26z ...
cn_600022%2Ccn_600516%2Ccn_000002%2Ccn_600519% ...
_%3D1498179095094%26list%3Dsh600030 ...


In [7]:
evil.show()

Canvas is accessible via web browser at the URL: http://localhost:57383/index.html
Opening Canvas in default web browser.


In [4]:
evil.head()

param
Search%3D%3C/script%3E%3Cimg/%2A%00/src%3D%22w ...
symbol%3D%3Ch1%3E%3Cscript%3Ealert%28/hacked/% ...
query%3D%3CIMG%2B%22%22%22%3E%3CSCRIPT%3Ealert ...
ReturnUrl%3Dhttp%3A//www.elle.fr/recherche ...
_lang%3D%22%3E%3Cscript%3Ealert%28document.coo ...
language%3D%22%3E%3C/script%3E%22%3E%27%3E%3Cs ...
q%3Dbentley%26stylesheet%3D%22%3E%3Cscript%3Ea ...
option%3Dcom_wdshop%26view%3Duserinfo%26ajax_j ...
CT_ORIG_URL%3D/arena/%22%3E%3Cscript%3Ealert%2 ...
query%3DSearch...%26Product%3D%27%22--%3E%3C/s ...


# 3 范化、分词

In [3]:
import nltk
import re
from urllib import unquote

In [4]:
def GeneSeg(payload):
    #数字泛化为"0"
    payload=payload['param'].lower()
    payload=unquote(unquote(payload))
    payload,num=re.subn(r'\d+',"0",payload)
    #替换url为”http://u
    payload,num=re.subn(r'(http|https)://[a-zA-Z0-9\.@&/#!#\?]+', "http://u", payload)
    #分词
    r = '''
        (?x)[\w\.]+?\(
        |\)
        |"\w+?"
        |'\w+?'
        |http://\w
        |</\w+>
        |<\w+>
        |<\w+
        |\w+=
        |>
        |[\w\.]+
    '''
    return nltk.regexp_tokenize(payload, r)

In [5]:
# 增加一列
normal['parsed'] = normal.apply(GeneSeg)
evil['parsed'] = evil.apply(GeneSeg)

In [6]:
normal.head()

param,parsed
_%3D1498591621808,"[_=, 0]"
code%3Dzs_000001%2Czs_399001%2Czs_399006%26cb% ...,"[code=, zs_0, zs_0, zs_0,cb=, fortune_hq_cn, _=, ..."
_%3D1498591951848%26list%3Dml_sh600030 ...,"[_=, 0, list=, ml_sh0]"
6053%26ri%3Dzb6-00f%7E-04gUry-01h- ...,"[0, ri=, zb0, 0f, 0gury,0h, 0rc, tn=, 0, en=, ..."
b1498592370545%3D1,"[b0=, 0]"
v%3D13111002,"[v=, 0]"
COLLCC%3D3442798258%26,"[collcc=, 0]"
t%3Dcheck%26rec%3Dstratus%26etyp%3Dconnect%26z ...,"[t=, check, rec=,stratus, etyp=, connect, ..."
cn_600022%2Ccn_600516%2Ccn_000002%2Ccn_600519% ...,"[cn_0, cn_0, cn_0, cn_0,cn_0, cn_0, cn_0, cn_0, ..."
_%3D1498179095094%26list%3Dsh600030 ...,"[_=, 0, list=, sh0]"


In [7]:
evil.head()

param,parsed
Search%3D%3C/script%3E%3Cimg/%2A%00/src%3D%22w ...,"[search=, </script>,<img, src=, ..."
symbol%3D%3Ch1%3E%3Cscript%3Ealert%28/hacked/% ...,"[symbol=, <h0>, <script>,alert(, hacked, ), ..."
query%3D%3CIMG%2B%22%22%22%3E%3CSCRIPT%3Ealert ...,"[query=, <img, >,<script>, alert(, ..."
ReturnUrl%3Dhttp%3A//www.elle.fr/recherche ...,"[returnurl=, http://u,globale, searchtext, ), ..."
_lang%3D%22%3E%3Cscript%3Ealert%28document.coo ...,"[_lang=, >, <script>,alert(, document.cookie, ..."
language%3D%22%3E%3C/script%3E%22%3E%27%3E%3Cs ...,"[language=, >, </script>,>, >, <script>, alert(, ..."
q%3Dbentley%26stylesheet%3D%22%3E%3Cscript%3Ea ...,"[q=, bentley,stylesheet=, >, <scri ..."
option%3Dcom_wdshop%26view%3Duserinfo%26ajax_j ...,"[option=, com_wdshop,view=, userinfo, ..."
CT_ORIG_URL%3D/arena/%22%3E%3Cscript%3Ealert%2 ...,"[ct_orig_url=, arena, >,<script>, alert(, 0, ), ..."
query%3DSearch...%26Product%3D%27%22--%3E%3C/s ...,"[query=, search...,product=, >, </style>, ..."


# 4 词表

In [8]:
# 取词的全集生成新的SFrame
ans = []
evil['parsed'].apply(lambda x:[ans.append(i) for i in x])
print ans

['search=', '</script>', '<img', 'src=', 'worksinchrome', 'colon', 'prompt', 'x0', '0', 'x0', 'onerror=', 'eval(', 'src', ')', '>', 'symbol=', '<h0>', '<script>', 'alert(', 'hacked', ')', '</script>', '</h0>', 'id=', '0', 'query=', '<img', '>', '<script>', 'alert(', '"cyb0r_pr0dat0r"', ')', '</script>', '>', 'fuzzy=', 'true', 'ct=', 'null', 'autobounce=', 'true', 'returnurl=', 'http://u', 'globale', 'searchtext', ')', '>', '<script>', 'alert(', 'xss', 'by', 'm0ke', ')', '</script>', '_lang=', '>', '<script>', 'alert(', 'document.cookie', ')', '</script>', 'language=', '>', '</script>', '>', '>', '<script>', 'alert(', 'arman', 'ires', 'digital', 'boys', 'underground', 'http://u', ')', '</script>', 'q=', 'bentley', 'stylesheet=', '>', '<script>', 'alert(', "'raven'", ')', '</script>', 'collections=', 'libcms', 'option=', 'com_wdshop', 'view=', 'userinfo', 'ajax_json=', 'ajax_fill_city_state', 'format=', 'raw', 'zip=', '>', '</style>', '</script>', '<script>', 'alert(', 'document.cookie',

In [9]:
evil_word_frame = gl.SFrame(data=ans)

In [10]:
evil_word_frame

X1
search=
</script>
<img
src=
worksinchrome
colon
prompt
x0
0
x0


In [12]:
# 增加统计列
evil_word_frame['count'] = evil_word_frame.apply(lambda x:len(evil_word_frame.filter_by(x['X1'],'X1')))

In [13]:
# 去重，按数量排序
evil_word_frame = evil_word_frame.unique().sort('count',ascending=False)

In [14]:
evil_word_frame

X1,count
0,491
),155
>,120
alert(,113
</script>,99
<script>,83
string.fromcharcode(,37
http://u,15
document.cookie,15
xss,10


In [16]:
# 取前300个词作为词表
wordlist_len = 300

evil_word_frame = evil_word_frame[0:wordlist_len]

In [17]:
evil_word_frame

X1,count
0,491
),155
>,120
alert(,113
</script>,99
<script>,83
string.fromcharcode(,37
http://u,15
document.cookie,15
xss,10


In [18]:
evil_word_frame.save('file/evil_word_list.csv', format='csv')

# 5 词向量

In [19]:
# 将不在词表内的词全部替换为'WORD'，结果存入'words'列
evil['words'] = evil.apply(lambda x:[i if len(evil_word_frame.filter_by(i,'X1')) else 'WORD' for i in x['parsed']])

In [20]:
evil

param,parsed,words
Search%3D%3C/script%3E%3Cimg/%2A%00/src%3D%22w ...,"[search=, </script>,<img, src=, ...","[search=, </script>,<img, src=, ..."
symbol%3D%3Ch1%3E%3Cscript%3Ealert%28/hacked/% ...,"[symbol=, <h0>, <script>,alert(, hacked, ), ...","[WORD, <h0>, <script>,alert(, hacked, ), ..."
query%3D%3CIMG%2B%22%22%22%3E%3CSCRIPT%3Ealert ...,"[query=, <img, >,<script>, alert(, ...","[query=, <img, >,<script>, alert(, ..."
ReturnUrl%3Dhttp%3A//www.elle.fr/recherche ...,"[returnurl=, http://u,globale, searchtext, ), ...","[returnurl=, http://u,globale, searchtext, ), ..."
_lang%3D%22%3E%3Cscript%3Ealert%28document.coo ...,"[_lang=, >, <script>,alert(, document.cookie, ...","[_lang=, >, <script>,alert(, document.cookie, ..."
language%3D%22%3E%3C/script%3E%22%3E%27%3E%3Cs ...,"[language=, >, </script>,>, >, <script>, alert(, ...","[language=, >, </script>,>, >, <script>, alert(, ..."
q%3Dbentley%26stylesheet%3D%22%3E%3Cscript%3Ea ...,"[q=, bentley,stylesheet=, >, <scri ...","[q=, bentley,stylesheet=, >, <scri ..."
option%3Dcom_wdshop%26view%3Duserinfo%26ajax_j ...,"[option=, com_wdshop,view=, userinfo, ...","[WORD, com_wdshop, view=,userinfo, ajax_json=, ..."
CT_ORIG_URL%3D/arena/%22%3E%3Cscript%3Ealert%2 ...,"[ct_orig_url=, arena, >,<script>, alert(, 0, ), ...","[ct_orig_url=, arena, >,<script>, alert(, 0, ), ..."
query%3DSearch...%26Product%3D%27%22--%3E%3C/s ...,"[query=, search...,product=, >, </style>, ...","[query=, search...,product=, >, </style>, ..."


In [21]:
normal['words'] = normal.apply(lambda x:[i if len(evil_word_frame.filter_by(i,'X1')) else 'WORD' for i in x['parsed']])

In [22]:
normal

param,parsed,words
_%3D1498591621808,"[_=, 0]","[WORD, 0]"
code%3Dzs_000001%2Czs_399001%2Czs_399006%26cb% ...,"[code=, zs_0, zs_0, zs_0,cb=, fortune_hq_cn, _=, ...","[WORD, WORD, WORD, WORD,WORD, WORD, WORD, 0] ..."
_%3D1498591951848%26list%3Dml_sh600030 ...,"[_=, 0, list=, ml_sh0]","[WORD, 0, WORD, WORD]"
6053%26ri%3Dzb6-00f%7E-04gUry-01h- ...,"[0, ri=, zb0, 0f, 0gury,0h, 0rc, tn=, 0, en=, ...","[0, WORD, WORD, WORD,WORD, WORD, WORD, WORD, ..."
b1498592370545%3D1,"[b0=, 0]","[WORD, 0]"
v%3D13111002,"[v=, 0]","[v=, 0]"
COLLCC%3D3442798258%26,"[collcc=, 0]","[WORD, 0]"
t%3Dcheck%26rec%3Dstratus%26etyp%3Dconnect%26z ...,"[t=, check, rec=,stratus, etyp=, connect, ...","[WORD, WORD, WORD, WORD,WORD, WORD, WORD, WORD, ..."
cn_600022%2Ccn_600516%2Ccn_000002%2Ccn_600519% ...,"[cn_0, cn_0, cn_0, cn_0,cn_0, cn_0, cn_0, cn_0, ...","[WORD, WORD, WORD, WORD,WORD, WORD, WORD, WORD, ..."
_%3D1498179095094%26list%3Dsh600030 ...,"[_=, 0, list=, sh0]","[WORD, 0, WORD, WORD]"


In [ ]:
# gl.SFrame([evil['words']]).save('file/evil.csv', format='csv')

In [ ]:
# from graphlab.toolkits.feature_engineering import OneHotEncoder
# encoder = gl.feature_engineering.create(evil, OneHotEncoder(features = ['words']))

In [ ]:
# transformed_sf = encoder.transform(sf)

## 5.1 word2vec模型训练

In [23]:
from gensim.models.word2vec import Word2Vec

Using TensorFlow backend.


In [24]:
embedding_size=128
skip_window=5
num_sampled=64
num_iter=100
data_set = evil['words']

In [25]:
data_set[0:10]

dtype: list
Rows: 10
[['search=', '</script>', '<img', 'src=', 'worksinchrome', 'colon', 'prompt', 'x0', '0', 'x0', 'onerror=', 'eval(', 'src', ')', '>'], ['WORD', '<h0>', '<script>', 'alert(', 'hacked', ')', '</script>', '</h0>', 'id=', '0'], ['query=', '<img', '>', '<script>', 'alert(', '"cyb0r_pr0dat0r"', ')', '</script>', '>', 'WORD', 'true', 'ct=', 'null', 'autobounce=', 'true'], ['returnurl=', 'http://u', 'globale', 'searchtext', ')', '>', '<script>', 'alert(', 'xss', 'by', 'm0ke', ')', '</script>'], ['_lang=', '>', '<script>', 'alert(', 'document.cookie', ')', '</script>'], ['language=', '>', '</script>', '>', '>', '<script>', 'alert(', 'arman', 'ires', 'digital', 'boys', 'underground', 'http://u', ')', '</script>'], ['q=', 'bentley', 'stylesheet=', '>', '<script>', 'alert(', "'raven'", ')', '</script>', 'collections=', 'libcms'], ['WORD', 'com_wdshop', 'view=', 'userinfo', 'ajax_json=', 'ajax_fill_city_state', 'format=', 'WORD', 'zip=', '>', '</style>', '</script>', '<script>',

In [26]:
# 时间久
model=Word2Vec(data_set,size=embedding_size,window=skip_window,negative=num_sampled,iter=num_iter)
#model=Word2Vec(data_set)

## 5.2 模型存取

In [27]:
model.save('model_word2vec_auto')

# import pickle
# with open('model_word2vec_pickle','wb') as f:
#     pickle.dump(model,f)

In [28]:
model_new = Word2Vec.load('model_word2vec_auto')

# import pickle
# model_new = pickle.load('model_word2vec_pickle')

## 5.3 数据存取

In [ ]:
# evil.save('evil_data')
# normal.save('normal_data')

In [ ]:
# evil = graphlab.SFrame('evil_data')
# normal = graphlab.SFrame('normal_data')

## 5.3 测试模型

In [29]:
embeddings=model_new.wv

In [32]:
embeddings.similar_by_word("</script>",5)

[('keyword=', 0.4176161289215088),
 ('searchfor=', 0.39741984009742737),
 ('page=', 0.38440901041030884),
 ('id=', 0.3829260766506195),
 ('q=', 0.3615732789039612)]

In [33]:
embeddings.similar_by_word("alert(",5)

[(')', 0.3255831003189087),
 ('</marquee>', 0.3212870955467224),
 ('</script>', 0.3093520998954773),
 ('</h0>', 0.30848926305770874),
 ('<h0>', 0.2897389233112335)]

## 5.4 添加向量和标签

In [34]:
dictionary=dict([(embeddings.index2word[i],i)for i in range(len(embeddings.index2word))])
reverse_dictionary=dict(zip(dictionary.values(),dictionary.keys()))

In [35]:
word2vec={"dictionary":dictionary,"embeddings":embeddings,"reverse_dictionary":reverse_dictionary}

In [36]:
from graphlab import SArray

def generate_vec(words):
    l = SArray([0.0]*128)
    for word in words:
        if word in dictionary:
            l += SArray(embeddings[word])
    return l

In [37]:
# 添加新的vec列、label列
black = gl.SFrame([evil.apply(lambda x:generate_vec(x['words'])),evil.apply(lambda x:1)])
white = gl.SFrame([normal.apply(lambda x:generate_vec(x['words'])),normal.apply(lambda x:0)])

In [38]:
black.show()

Canvas is accessible via web browser at the URL: http://localhost:59854/index.html
Opening Canvas in default web browser.


In [40]:
#black.save('fuck_black')
#white.save('fuck_white')
len(white)

200129

# 6 神经网络

## 6.1 数据拆分

In [41]:
black_train,black_test = black.random_split(0.8,seed=0)
white_train,white_test = white.random_split(0.8,seed=0)

In [42]:
len(black_train)

32543

In [43]:
data_train = black_train.append(white_train)
data_test = black_test.append(white_test)

In [ ]:
print len(data_train),len(data_test)

## 6.2 MLP

In [31]:
import keras
from keras.models import Sequential
from keras.layers import Dense,InputLayer,Dropout,Flatten
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
from keras.models import load_model

In [32]:
batch_size=500
epochs_num=1
log_dir="log/MLP.log"
model_dir="file/MLP_model"

In [34]:
def train(train_generator,train_size,input_num,dims_num):
    print("Start Train Job! ")
    start=time.time()
    inputs=InputLayer(input_shape=(input_num,dims_num),batch_size=batch_size)
    layer1=Dense(100,activation="relu")
    layer2=Dense(20,activation="relu")
    flatten=Flatten()
    layer3=Dense(2,activation="softmax",name="Output")
    optimizer=Adam()
    model=Sequential()
    model.add(inputs)
    model.add(layer1)
    model.add(Dropout(0.5))
    model.add(layer2)
    model.add(Dropout(0.5))
    model.add(flatten)
    model.add(layer3)
    call=TensorBoard(log_dir=log_dir,write_grads=True,histogram_freq=1)
    model.compile(optimizer,loss="categorical_crossentropy",metrics=["accuracy"])
    model.fit_generator(train_generator,steps_per_epoch=train_size//batch_size,epochs=epochs_num,callbacks=[call])
#    model.fit_generator(train_generator, steps_per_epoch=5, epochs=5, callbacks=[call])
    model.save(model_dir)
    end=time.time()
    print("Over train job in %f s"%(end-start))

In [35]:
train(data_train,len(data_train),len(data_train),embeddings["WORD"].shape[0])

128